In [11]:
import os
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
utils = importr("utils")
RRA = importr("RobustRankAggreg")
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from utils import *

In [12]:
with open(file="cv_outer.pickle", mode="rb") as file:
    cv_outer = joblib.load(file)
df = pd.read_csv("df.csv", index_col=0)
X = df.drop(["target", "dataset"], axis=1).copy()
y = df["target"].replace({"Bystander": 1, "Control": 0})
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X_devs = []
y_devs = []
X_tests = []
y_tests = []
for i, outer in enumerate(cv_outer):
    dev_idx, test_idx = outer
    X_devs.append(X.iloc[dev_idx, :].copy())
    y_devs.append(y.iloc[dev_idx].copy())
    X_tests.append(X.iloc[test_idx, :].copy())
    y_tests.append(y.iloc[test_idx].copy())

In [13]:
models_best = {
    "Dummy_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("Dummy_tuned") and study_pkl.endswith("best.pkl")
    ],
    "LR_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("LR_tuned") and study_pkl.endswith("best.pkl")
    ],
    "SVC_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("SVC_tuned") and study_pkl.endswith("best.pkl")
    ],
    "KNN_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("KNN_tuned") and study_pkl.endswith("best.pkl")
    ],
    "GP_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("GP_tuned") and study_pkl.endswith("best.pkl")
    ],
    "GNB_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("GNB_tuned") and study_pkl.endswith("best.pkl")
    ],
    "TN_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("TN_tuned") and study_pkl.endswith("best.pkl")
    ],
    "AB_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("AB_tuned") and study_pkl.endswith("best.pkl")
    ],
    "GB_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("GB_tuned") and study_pkl.endswith("best.pkl")
    ],
    "XGB_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("XGB_tuned") and study_pkl.endswith("best.pkl")
    ],
    "LGBM_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("LGBM_tuned") and study_pkl.endswith("best.pkl")
    ],
    "CB_tuned": [
        joblib.load(study_pkl)
        for study_pkl in os.listdir()
        if study_pkl.startswith("CB_tuned") and study_pkl.endswith("best.pkl")
    ],
}

In [14]:
evaluation_classes = {
    key: Evaluation(
        value, X_tests, y_tests, cv=cv, tag=key, X_devs=X_devs, y_devs=y_devs
    )
    for (key, value) in models_best.items()
}

In [15]:
metrics_merged = pd.concat(
    [eval_class.compute_metrics() for eval_class in evaluation_classes.values()],
    ignore_index=True,
    axis=0,
)

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zer

In [16]:
metrics_merged = metrics_merged[metrics_merged.algorithm != "Dummy_tuned"]
metrics_merged = metrics_merged.loc[
    metrics_merged.groupby(["algorithm"])["test_ROC"].idxmax()
]
best_algorithm_fold = metrics_merged[["algorithm", "fold"]]
best_algorithm_fold = best_algorithm_fold.set_index("algorithm")

In [17]:
# SHAP
LR_gene_rank = evaluation_classes["LR_tuned"].get_shap_ranks(best_algorithm_fold)
SVC_gene_rank = evaluation_classes["SVC_tuned"].get_shap_ranks(best_algorithm_fold)
KNN_gene_rank = evaluation_classes["KNN_tuned"].get_shap_ranks(best_algorithm_fold)
GP_gene_rank = evaluation_classes["GP_tuned"].get_shap_ranks(best_algorithm_fold)
GNB_gene_rank = evaluation_classes["GNB_tuned"].get_shap_ranks(best_algorithm_fold)
TN_gene_rank = evaluation_classes["TN_tuned"].get_shap_ranks(best_algorithm_fold)
AB_gene_rank = evaluation_classes["AB_tuned"].get_shap_ranks(best_algorithm_fold)
GB_gene_rank = evaluation_classes["GB_tuned"].get_shap_ranks(best_algorithm_fold)
XGB_gene_rank = evaluation_classes["XGB_tuned"].get_shap_ranks(best_algorithm_fold)
LGBM_gene_rank = evaluation_classes["LGBM_tuned"].get_shap_ranks(best_algorithm_fold)
CB_gene_rank = evaluation_classes["CB_tuned"].get_shap_ranks(best_algorithm_fold)

Using 194 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


100%|██████████| 194/194 [06:41<00:00,  2.07s/it]
Using 194 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 194/194 [20:33<00:00,  6.36s/it]
Using 194 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 194/194 [15:13<00:00,  4.71s/it]
Using 194 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 194/194 [1:22:50<00:00, 25.62s/it]
Using 194 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 194/194 [07:32<00:00,  2.33s/it]
Using 194 background data samples could cause slower run times. Consider using shap.

In [20]:
# RRA
gene_list_20 = [
    AB_gene_rank.feature[:20],
    GNB_gene_rank.feature[:20],
    TN_gene_rank.feature[:20],
    XGB_gene_rank.feature[:20],
    LGBM_gene_rank.feature[:20],
    GB_gene_rank.feature[:20],
    CB_gene_rank.feature[:20],
    LR_gene_rank.feature[:20],
    KNN_gene_rank.feature[:20],
    GP_gene_rank.feature[:20],
    SVC_gene_rank.feature[:20],
]

In [21]:
gene_rank_R_20 = RRA.aggregateRanks(gene_list_20)
gene_rank_py_20 = robjects.conversion.rpy2py(gene_rank_R_20)
gene_rank_py_20.to_csv("gene_rank_rra_20.csv", index=False)